In [1]:
import pandas as pd
import os
import geopandas as gpd
from pymongo import MongoClient
import requests
import json 
from dotenv import load_dotenv
import folium
from folium import Choropleth, Circle, Marker, Icon, Map

In [2]:
client = MongoClient("mongodb://localhost/ironhack")
db = client.get_database()

In [5]:
list(db.list_collections())

[{'name': 'companies',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('f0df70fa-ecbb-4210-b11a-d2f3d3075dc8')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [6]:
db.companies.distinct("category_code") 

[None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web']

I make a dataframe with all of the web companies, I use London because is one of the European cities with the most tech companies.

In [21]:
query1 = {"category_code":"web", "offices.city": {"$eq":"London"}}
project = {"_id":0, "offices.city":1, "offices.latitude":1, "offices.longitude":1}
res = db.companies.find(query1,project)

In [22]:
res

In [23]:
df_loc = list(res)
df_loc = pd.DataFrame.from_dict(df_loc)

In [24]:
df_loc

,offices
0,"[{'city': 'San Jose', 'latitude': 37.321798, '..."
1,"[{'city': 'Los Angeles', 'latitude': 37.09024,..."
2,"[{'city': 'London', 'latitude': 37.09024, 'lon..."
3,"[{'city': 'London', 'latitude': 51.5192042, 'l..."
4,"[{'city': 'London', 'latitude': 37.09024, 'lon..."
...,...
107,"[{'city': 'London', 'latitude': 51.5215279, 'l..."
108,"[{'city': 'London', 'latitude': None, 'longitu..."
109,"[{'city': 'London', 'latitude': None, 'longitu..."
110,"[{'city': '', 'latitude': None, 'longitude': N..."


In [35]:
query = {"category_code":"web"}
project = {"_id":0, "name":1, "category_code":1}

res = db.companies.find(query,project)
res = list(res)
res[:5]

[{'name': 'Wetpaint', 'category_code': 'web'},
 {'name': 'Postini', 'category_code': 'web'},
 {'name': 'Geni', 'category_code': 'web'},
 {'name': 'Fox Interactive Media', 'category_code': 'web'},
 {'name': 'StumbleUpon', 'category_code': 'web'}]

In [36]:
df_res = pd.DataFrame(res)

In [37]:
df_res

,name,category_code
0,Wetpaint,web
1,Postini,web
2,Geni,web
3,Fox Interactive Media,web
4,StumbleUpon,web
...,...,...
3782,Optemo,web
3783,Zoomission,web
3784,Eazeeloans,web
3785,DocASAP,web


In [51]:
result = pd.concat([df_res, df_loc], axis=1)

In [52]:
result

,name,category_code,offices
0,Wetpaint,web,"[{'city': 'San Jose', 'latitude': 37.321798, '..."
1,Postini,web,"[{'city': 'Los Angeles', 'latitude': 37.09024,..."
2,Geni,web,"[{'city': 'London', 'latitude': 37.09024, 'lon..."
3,Fox Interactive Media,web,"[{'city': 'London', 'latitude': 51.5192042, 'l..."
4,StumbleUpon,web,"[{'city': 'London', 'latitude': 37.09024, 'lon..."
...,...,...,...
3782,Optemo,web,NaN
3783,Zoomission,web,NaN
3784,Eazeeloans,web,NaN
3785,DocASAP,web,NaN


In [53]:
result1 = result.explode('offices')
result1

,name,category_code,offices
0,Wetpaint,web,"{'city': 'San Jose', 'latitude': 37.321798, 'l..."
0,Wetpaint,web,"{'city': 'London', 'latitude': 51.500152, 'lon..."
1,Postini,web,"{'city': 'Los Angeles', 'latitude': 37.09024, ..."
1,Postini,web,"{'city': 'London', 'latitude': None, 'longitud..."
1,Postini,web,"{'city': 'New York', 'latitude': None, 'longit..."
...,...,...,...
3782,Optemo,web,NaN
3783,Zoomission,web,NaN
3784,Eazeeloans,web,NaN
3785,DocASAP,web,NaN


In [88]:
full_address = result1[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)

In [89]:
full_address

,city,latitude,longitude
0,San Jose,37.321798,-121.979453
0,London,51.500152,-0.126236
1,Los Angeles,37.090240,-95.712891
1,London,NaN,NaN
1,New York,NaN,NaN
...,...,...,...
3782,NaN,NaN,NaN
3783,NaN,NaN,NaN
3784,NaN,NaN,NaN
3785,NaN,NaN,NaN


In [136]:
join = pd.concat([result1, full_address], axis=1)

In [137]:
join

,name,category_code,offices,city,latitude,longitude
0,Wetpaint,web,"{'city': 'San Jose', 'latitude': 37.321798, 'l...",San Jose,37.321798,-121.979453
0,Wetpaint,web,"{'city': 'London', 'latitude': 51.500152, 'lon...",London,51.500152,-0.126236
1,Postini,web,"{'city': 'Los Angeles', 'latitude': 37.09024, ...",Los Angeles,37.090240,-95.712891
1,Postini,web,"{'city': 'London', 'latitude': None, 'longitud...",London,NaN,NaN
1,Postini,web,"{'city': 'New York', 'latitude': None, 'longit...",New York,NaN,NaN
...,...,...,...,...,...,...
3782,Optemo,web,NaN,NaN,NaN,NaN
3783,Zoomission,web,NaN,NaN,NaN,NaN
3784,Eazeeloans,web,NaN,NaN,NaN,NaN
3785,DocASAP,web,NaN,NaN,NaN,NaN


In [142]:
join = join.drop(columns = "offices")

In [143]:
join.loc[join.city == 'London', 'city'].count()


112

In [144]:
join.city.unique()

array(['San Jose', 'London', 'Los Angeles', 'New York', 'Chicago',
       'Toronto', 'Singapore', 'Zurich', 'Chiyoda-ku Tokyo',
       'San Fransisco', 'Moscow', 'Munich', 'Hong Kong', 'Paris',
       'San Francisco', 'Sydney', 'Montreal', 'Ljubljana', 'Stockholm',
       '', 'Madrid', 'Hamburg', 'Lod', 'Cologne', 'Celebration',
       'New York City', 'Beijing', 'Dublin', 'Waterford', 'Edinburgh',
       'Cambridge', 'Christchurch', 'Houston', 'Atlanta', 'Dallas',
       'Denver', 'Costa Mesa', 'Ft. Lauderdale', 'Seattle', 'Herndon',
       'Tokyo', 'Jersey City', 'Chao Yang, Beijing', 'Milton Keynes', nan],
      dtype=object)

In [153]:
Lnd_offices = join[(join.city != 'San Jose') & (join.city != 'Los Angeles') & (join.city != 'New York') & (join.city != 'Chicago') & (join.city != 'Toronto') & (join.city != 'Singapore') & (join.city != 'Zurich') & (join.city != 'Chiyoda-ku Tokyo') & (join.city != 'San Francisco') & (join.city != 'Moscow') & (join.city != 'Munich') & (join.city != 'Hong Kong')  & (join.city != 'Paris')  & (join.city != 'Sydney') & (join.city != 'Montreal')  & (join.city != 'Ljubljana') & (join.city != 'Stockholm')  & (join.city != '')  & (join.city != 'Madrid')  & (join.city != 'Hamburg') & (join.city != 'Lod') & (join.city != 'Cologne') & (join.city != 'Celebration') & (join.city != 'New York City') & (join.city != 'Beijing')  & (join.city != 'Dublin') & (join.city != 'Waterford') & (join.city != 'Edinburgh')  & (join.city != 'Cambridge') & (join.city != 'Christchurch') & (join.city != 'Houston') & (join.city != 'Atlanta') & (join.city != 'Dallas') & (join.city != 'Denver') & (join.city != 'Costa Mesa') & (join.city != 'Ft. Lauderdale') & (join.city != 'Seattle') & (join.city != 'Herndon') & (join.city != 'Tokyo') & (join.city != 'Jersey City') & (join.city != 'Chao Yang') & (join.city != 'Beijing') & (join.city != 'Milton Keynes')] 

In [154]:
Lnd_offices = Lnd_offices.dropna()
Lnd_offices

,name,category_code,city,latitude,longitude
0,Wetpaint,web,London,51.500152,-0.126236
2,Geni,web,London,37.090240,-95.712891
3,Fox Interactive Media,web,London,51.519204,-0.162610
4,StumbleUpon,web,London,37.090240,-95.712891
5,Gizmoz,web,London,51.523789,-0.087432
...,...,...,...,...,...
103,Tangler,web,London,51.511444,-0.135691
105,BillMonk,web,London,51.514735,-0.087424
106,TalkPlus,web,London,51.579373,-0.116242
107,MeeMix,web,London,51.521528,-0.103822


In [155]:
Lnd_offices.city.unique()

array(['London'], dtype=object)

I will choose an office and see if it could be an appropiate location for our new headquarters

In [209]:
print(Lnd_offices.loc[0])


name              Wetpaint
category_code          web
city                London
latitude         51.500152
longitude        -0.126236
Name: 0, dtype: object


In [210]:
office_lat = 51.500152
office_long = -0.126236

In [221]:
office = [("office","51.500152","-0.126236", "POINT (-0.126236 51.500152)")]
df_office = pd.DataFrame(office) 
df_office

,0,1,2,3
0,office,51.500152,-0.126236,POINT (-0.126236 51.500152)


In [222]:
df_office.columns = ['name', 'latitud', 'longitud', 'geometry'] 
df_office

,name,latitud,longitud,geometry
0,office,51.500152,-0.126236,POINT (-0.126236 51.500152)


In [223]:
map_1 = folium.Map(location = [office_lat,office_long], zoom_start=14)
map_1

In [224]:
icono = Icon(color = "red",
             prefix = "fa",
             icon = "home",
             icon_color = "black",
             tooltip = "Possible location")

loc = {"location":[office_lat,office_long],
      "tooltip": "Possible location"}

marker = Marker(**loc, icon = icono)

marker.add_to(map_1)
map_1

In [225]:
type(office_location)


list

# Transportation

London is a busy city, with a lot of traffic. It is important for the account managers and other workers to be near the Tube station. 

In [78]:
empresas1 = {"founded_year":{"$lte":2010}}

In [79]:
listaempresas1 = list(db["companies"].find(empresas1,{"_id":0,"name":1,"founded_year":1,"offices":1}))


In [80]:
df_empresas1 = pd.DataFrame(listaempresas1)


In [81]:
df_empresas1 = df_empresas1.explode('offices')


In [82]:
df_empresas1

,name,founded_year,offices
0,Wetpaint,2005,"{'description': '', 'address1': '710 - 2nd Ave..."
0,Wetpaint,2005,"{'description': '', 'address1': '270 Lafayette..."
1,AdventNet,1996,"{'description': 'Headquarters', 'address1': '4..."
2,Zoho,2005,"{'description': 'Headquarters', 'address1': '4..."
3,Digg,2004,"{'description': None, 'address1': '135 Mississ..."
...,...,...,...
13045,QSGI,1967,"{'description': '', 'address1': '400 Royal Pal..."
13046,Oriact,2009,"{'description': '', 'address1': 'LÃ¤rchenweg 7..."
13047,Adhunk,2009,"{'description': 'Indian Office', 'address1': '..."
13048,AfterLogic,2002,"{'description': 'Livingston', 'address1': 'P.O..."


In [83]:
dfOfficeData = df_empresas1[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)

In [104]:
dfOfficeData

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
1,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
3,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
...,...,...,...,...,...,...,...,...,...
13045,,400 Royal Palm Way,,33480,Palm Beach,FL,USA,26.705331,-80.041395
13046,,LÃ¤rchenweg 7,,6403,KÃ¼ssnacht,None,CHE,47.088219,8.437163
13047,Indian Office,Near Tube Well(south),Patel Nagar,334003,Bikaner,None,IND,NaN,NaN
13048,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [110]:
fin_empresas1 = pd.concat([df_empresas1,dfOfficeData], axis=1)

In [111]:
fin_empresas1 = fin_empresas1.drop(columns=["offices"])

In [112]:
fin_empresas1["location"] = fin_empresas1[["latitude","longitude"]].apply(lambda x:fun.asGeoJSON(x.latitude,x.longitude), axis=1)

NameError: name 'fun' is not defined

In [108]:
fin_empresas1.to_json("output/empmasnue", orient="records")


FileNotFoundError: [Errno 2] No such file or directory: 'output/empmasnue'